# Exercice 2 : Identification et Traitement des Valeurs Manquantes

## Objectifs
- Identifier les valeurs manquantes dans différents types de datasets
- Explorer les patterns de valeurs manquantes
- Implémenter différentes stratégies de traitement
- Comparer l'impact des stratégies sur les performances

## Stratégies couvertes
1. **Suppression** : Lignes et colonnes
2. **Imputation simple** : Moyenne, médiane, mode
3. **Imputation avancée** : KNN, itérative (MICE)
4. **Analyse d'impact** : Comparaison des performances

## 1. Import des librairies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Configuration pour un meilleur affichage
pd.set_option('display.max_columns', None)
plt.style.use('default')
sns.set_palette("husl")

print("✅ Librairies importées avec succès!")

## 2. Création d'un dataset avec valeurs manquantes

In [ ]:
def create_sample_dataset():
    """Crée un dataset d'exemple avec différents types de valeurs manquantes"""
    np.random.seed(42)
    n_samples = 1000
    
    # Créer les données de base
    data = {
        'age': np.random.randint(18, 80, n_samples),
        'income': np.random.normal(50000, 15000, n_samples),
        'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_samples),
        'experience': np.random.randint(0, 40, n_samples),
        'city': np.random.choice(['Paris', 'Lyon', 'Marseille', 'Toulouse', 'Nice'], n_samples),
        'satisfaction': np.random.randint(1, 6, n_samples),
        'target': np.random.choice(['Classe_A', 'Classe_B', 'Classe_C'], n_samples)
    }
    
    df = pd.DataFrame(data)
    
    # Introduire différents types de valeurs manquantes
    
    # 1. MCAR (Missing Completely At Random) - Age
    mcar_indices = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
    df.loc[mcar_indices, 'age'] = np.nan
    
    # 2. MAR (Missing At Random) - Income manquant pour les jeunes
    young_indices = df[df['age'] < 25].index
    mar_indices = np.random.choice(young_indices, size=int(0.3 * len(young_indices)), replace=False)
    df.loc[mar_indices, 'income'] = np.nan
    
    # 3. MNAR (Missing Not At Random) - Education manquante pour satisfaction faible
    low_satisfaction = df[df['satisfaction'] <= 2].index
    mnar_indices = np.random.choice(low_satisfaction, size=int(0.4 * len(low_satisfaction)), replace=False)
    df.loc[mnar_indices, 'education'] = np.nan
    
    # 4. Valeurs manquantes additionnelles
    df.loc[np.random.choice(df.index, 50, replace=False), 'experience'] = np.nan
    df.loc[np.random.choice(df.index, 30, replace=False), 'city'] = np.nan
    
    return df

# Créer le dataset
df = create_sample_dataset()
print(f"Dataset créé: {df.shape}")
print(f"Valeurs manquantes totales: {df.isnull().sum().sum()}")

# Afficher les premières lignes
df.head(10)

## 3. Analyse exploratoire des valeurs manquantes

In [ ]:
# Statistiques générales
total_cells = np.product(df.shape)
total_missing = df.isnull().sum().sum()
missing_percentage = (total_missing / total_cells) * 100

print("=" * 60)
print("ANALYSE DES VALEURS MANQUANTES")
print("=" * 60)
print(f"Dimensions du dataset: {df.shape}")
print(f"Total de cellules: {total_cells:,}")
print(f"Total de valeurs manquantes: {total_missing:,}")
print(f"Pourcentage global: {missing_percentage:.2f}%")

# Statistiques par colonne
missing_by_column = df.isnull().sum()
missing_percentage_by_column = (missing_by_column / len(df)) * 100

missing_summary = pd.DataFrame({
    'Valeurs_Manquantes': missing_by_column,
    'Pourcentage': missing_percentage_by_column
}).sort_values('Valeurs_Manquantes', ascending=False)

print("\nValeurs manquantes par colonne:")
print(missing_summary[missing_summary['Valeurs_Manquantes'] > 0])

## 4. Visualisation des patterns de valeurs manquantes

In [ ]:
# Créer les visualisations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Analyse des Valeurs Manquantes', fontsize=16)

# 1. Heatmap des valeurs manquantes
missing_data = df.isnull()
sns.heatmap(missing_data, yticklabels=False, cbar=True, cmap='viridis', ax=axes[0, 0])
axes[0, 0].set_title('Heatmap des Valeurs Manquantes')

# 2. Barplot des valeurs manquantes par colonne
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=True)
missing_counts.plot(kind='barh', ax=axes[0, 1], color='coral')
axes[0, 1].set_title('Nombre de Valeurs Manquantes par Colonne')
axes[0, 1].set_xlabel('Nombre de valeurs manquantes')

# 3. Matrice de corrélation des patterns manquants
missing_corr = missing_data.corr()
sns.heatmap(missing_corr, annot=True, cmap='coolwarm', center=0, ax=axes[1, 0])
axes[1, 0].set_title('Corrélation des Patterns Manquants')

# 4. Distribution des valeurs manquantes par ligne
missing_per_row = missing_data.sum(axis=1)
missing_per_row.hist(bins=20, ax=axes[1, 1], color='skyblue')
axes[1, 1].set_title('Distribution des Valeurs Manquantes par Ligne')
axes[1, 1].set_xlabel('Nombre de valeurs manquantes par ligne')
axes[1, 1].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

## 5. Stratégie 1 : Suppression des lignes

In [ ]:
print("=" * 60)
print("STRATÉGIE 1: SUPPRESSION DES LIGNES")
print("=" * 60)

# Option A: Supprimer toutes les lignes avec des valeurs manquantes
df_drop_any = df.dropna()
print(f"Suppression de toutes les lignes avec valeurs manquantes:")
print(f"  Forme originale: {df.shape}")
print(f"  Forme après suppression: {df_drop_any.shape}")
print(f"  Lignes supprimées: {df.shape[0] - df_drop_any.shape[0]}")
print(f"  Pourcentage conservé: {(len(df_drop_any)/len(df))*100:.1f}%")

# Option B: Supprimer seulement les lignes avec plus de X valeurs manquantes
threshold = 2  # Supprimer si plus de 2 valeurs manquantes
df_drop_thresh = df.dropna(thresh=len(df.columns) - threshold)
print(f"\nSuppression des lignes avec plus de {threshold} valeurs manquantes:")
print(f"  Forme après suppression: {df_drop_thresh.shape}")
print(f"  Lignes supprimées: {df.shape[0] - df_drop_thresh.shape[0]}")
print(f"  Pourcentage conservé: {(len(df_drop_thresh)/len(df))*100:.1f}%")

# Vérifier les valeurs manquantes restantes
print(f"\nValeurs manquantes restantes (dropna): {df_drop_any.isnull().sum().sum()}")
print(f"Valeurs manquantes restantes (thresh): {df_drop_thresh.isnull().sum().sum()}")

## 6. Stratégie 2 : Suppression des colonnes

In [ ]:
print("=" * 60)
print("STRATÉGIE 2: SUPPRESSION DES COLONNES")
print("=" * 60)

# Définir le seuil de suppression (ex: 30% de valeurs manquantes)
threshold = 0.3

# Calculer le pourcentage de valeurs manquantes par colonne
missing_percentage = df.isnull().sum() / len(df)
columns_to_drop = missing_percentage[missing_percentage > threshold].index

print(f"Seuil de suppression: {threshold*100:.0f}% de valeurs manquantes")
print(f"Colonnes à supprimer: {list(columns_to_drop)}")

# Supprimer les colonnes
df_drop_cols = df.drop(columns=columns_to_drop)

print(f"\nForme originale: {df.shape}")
print(f"Forme après suppression: {df_drop_cols.shape}")
print(f"Colonnes conservées: {df_drop_cols.shape[1]}/{df.shape[1]}")

# Afficher les valeurs manquantes restantes
remaining_missing = df_drop_cols.isnull().sum()
print(f"\nValeurs manquantes restantes par colonne:")
print(remaining_missing[remaining_missing > 0])

## 7. Stratégie 3 : Imputation simple

In [ ]:
print("=" * 60)
print("STRATÉGIE 3: IMPUTATION SIMPLE")
print("=" * 60)

# Préparer les données
df_impute_simple = df.copy()

# Séparer les colonnes par type
numeric_cols = df_impute_simple.select_dtypes(include=[np.number]).columns
categorical_cols = df_impute_simple.select_dtypes(include=['object']).columns

print(f"Colonnes numériques: {list(numeric_cols)}")
print(f"Colonnes catégorielles: {list(categorical_cols)}")

# Imputation pour les colonnes numériques (moyenne)
if len(numeric_cols) > 0:
    imputer_numeric = SimpleImputer(strategy='mean')
    df_impute_simple[numeric_cols] = imputer_numeric.fit_transform(df_impute_simple[numeric_cols])
    print(f"\n✅ Imputation par moyenne appliquée aux colonnes numériques")

# Imputation pour les colonnes catégorielles (mode)
if len(categorical_cols) > 0:
    for col in categorical_cols:
        if df_impute_simple[col].isnull().any():
            mode_value = df_impute_simple[col].mode()[0] if not df_impute_simple[col].mode().empty else 'Unknown'
            df_impute_simple[col].fillna(mode_value, inplace=True)
    print(f"✅ Imputation par mode appliquée aux colonnes catégorielles")

# Vérification
remaining_missing = df_impute_simple.isnull().sum().sum()
print(f"\nValeurs manquantes restantes: {remaining_missing}")
print(f"Forme du dataset: {df_impute_simple.shape}")

# Comparaison des statistiques avant/après
print("\nComparaison des moyennes (colonnes numériques):")
for col in numeric_cols:
    original_mean = df[col].mean()
    imputed_mean = df_impute_simple[col].mean()
    print(f"  {col}: {original_mean:.2f} → {imputed_mean:.2f}")

## 8. Stratégie 4 : Imputation par médiane

In [ ]:
print("=" * 60)
print("STRATÉGIE 4: IMPUTATION PAR MÉDIANE")
print("=" * 60)

# Préparer les données
df_impute_median = df.copy()

# Imputation par médiane pour les colonnes numériques
if len(numeric_cols) > 0:
    imputer_median = SimpleImputer(strategy='median')
    df_impute_median[numeric_cols] = imputer_median.fit_transform(df_impute_median[numeric_cols])
    print(f"✅ Imputation par médiane appliquée aux colonnes numériques")

# Imputation par mode pour les colonnes catégorielles
if len(categorical_cols) > 0:
    for col in categorical_cols:
        if df_impute_median[col].isnull().any():
            mode_value = df_impute_median[col].mode()[0] if not df_impute_median[col].mode().empty else 'Unknown'
            df_impute_median[col].fillna(mode_value, inplace=True)
    print(f"✅ Imputation par mode appliquée aux colonnes catégorielles")

# Vérification
print(f"\nValeurs manquantes restantes: {df_impute_median.isnull().sum().sum()}")

# Comparaison moyenne vs médiane
print("\nComparaison Moyenne vs Médiane:")
for col in numeric_cols:
    original_median = df[col].median()
    mean_imputed = df_impute_simple[col].median()
    median_imputed = df_impute_median[col].median()
    print(f"  {col}:")
    print(f"    Original: {original_median:.2f}")
    print(f"    Après imputation moyenne: {mean_imputed:.2f}")
    print(f"    Après imputation médiane: {median_imputed:.2f}")

## 9. Stratégie 5 : Imputation KNN

In [ ]:
print("=" * 60)
print("STRATÉGIE 5: IMPUTATION KNN")
print("=" * 60)

# Préparer les données pour KNN (encoder les variables catégorielles)
df_for_knn = df.copy()
label_encoders = {}

# Encoder les variables catégorielles
for col in categorical_cols:
    le = LabelEncoder()
    # Gérer les valeurs manquantes en les remplaçant temporairement
    mask = df_for_knn[col].notna()
    if mask.sum() > 0:  # Si il y a des valeurs non-manquantes
        df_for_knn.loc[mask, col] = le.fit_transform(df_for_knn.loc[mask, col])
        label_encoders[col] = le

# Convertir tout en numérique
df_numeric = df_for_knn.apply(pd.to_numeric, errors='coerce')

# Appliquer l'imputation KNN
k_neighbors = 5
imputer_knn = KNNImputer(n_neighbors=k_neighbors)
df_imputed_knn = imputer_knn.fit_transform(df_numeric)

# Créer le DataFrame résultat
df_impute_knn = pd.DataFrame(df_imputed_knn, columns=df.columns, index=df.index)

# Décoder les variables catégorielles
for col, le in label_encoders.items():
    # Arrondir et convertir en entier pour le décodage
    encoded_values = df_impute_knn[col].round().astype(int)
    # S'assurer que les valeurs sont dans la plage valide
    encoded_values = np.clip(encoded_values, 0, len(le.classes_) - 1)
    df_impute_knn[col] = le.inverse_transform(encoded_values)

print(f"✅ Imputation KNN avec k={k_neighbors} appliquée")
print(f"Valeurs manquantes restantes: {df_impute_knn.isnull().sum().sum()}")
print(f"Forme du dataset: {df_impute_knn.shape}")

# Afficher quelques statistiques
print("\nComparaison des moyennes (avant/après KNN):")
for col in numeric_cols:
    original_mean = df[col].mean()
    knn_mean = df_impute_knn[col].mean()
    print(f"  {col}: {original_mean:.2f} → {knn_mean:.2f}")

## 10. Stratégie 6 : Imputation itérative (MICE)

In [ ]:
print("=" * 60)
print("STRATÉGIE 6: IMPUTATION ITÉRATIVE (MICE)")
print("=" * 60)

# Préparer les données (même processus que pour KNN)
df_for_mice = df.copy()
mice_encoders = {}

# Encoder les variables catégorielles
for col in categorical_cols:
    le = LabelEncoder()
    mask = df_for_mice[col].notna()
    if mask.sum() > 0:
        df_for_mice.loc[mask, col] = le.fit_transform(df_for_mice.loc[mask, col])
        mice_encoders[col] = le

# Convertir en numérique
df_numeric_mice = df_for_mice.apply(pd.to_numeric, errors='coerce')

# Appliquer l'imputation itérative
max_iter = 10
imputer_mice = IterativeImputer(max_iter=max_iter, random_state=42)
df_imputed_mice = imputer_mice.fit_transform(df_numeric_mice)

# Créer le DataFrame résultat
df_impute_mice = pd.DataFrame(df_imputed_mice, columns=df.columns, index=df.index)

# Décoder les variables catégorielles
for col, le in mice_encoders.items():
    encoded_values = df_impute_mice[col].round().astype(int)
    encoded_values = np.clip(encoded_values, 0, len(le.classes_) - 1)
    df_impute_mice[col] = le.inverse_transform(encoded_values)

print(f"✅ Imputation MICE avec max_iter={max_iter} appliquée")
print(f"Nombre d'itérations utilisées: {imputer_mice.n_iter_}")
print(f"Valeurs manquantes restantes: {df_impute_mice.isnull().sum().sum()}")

# Comparaison des moyennes
print("\nComparaison des moyennes (avant/après MICE):")
for col in numeric_cols:
    original_mean = df[col].mean()
    mice_mean = df_impute_mice[col].mean()
    print(f"  {col}: {original_mean:.2f} → {mice_mean:.2f}")

## 11. Comparaison de toutes les stratégies

In [ ]:
# Regrouper tous les résultats
strategies_results = {
    'Original': df,
    'Suppression_Lignes': df_drop_any,
    'Suppression_Colonnes': df_drop_cols,
    'Imputation_Moyenne': df_impute_simple,
    'Imputation_Médiane': df_impute_median,
    'Imputation_KNN': df_impute_knn,
    'Imputation_MICE': df_impute_mice
}

# Créer un résumé comparatif
comparison_data = []

for strategy_name, data in strategies_results.items():
    if data is not None and len(data) > 0:
        comparison_data.append({
            'Stratégie': strategy_name,
            'Nb_Observations': len(data),
            'Nb_Features': len(data.columns),
            'Valeurs_Manquantes': data.isnull().sum().sum(),
            'Pourcentage_Données_Conservées': (len(data) / len(df)) * 100,
            'Pourcentage_Features_Conservées': (len(data.columns) / len(df.columns)) * 100
        })

comparison_df = pd.DataFrame(comparison_data)

print("=" * 80)
print("COMPARAISON DE TOUTES LES STRATÉGIES")
print("=" * 80)
print(comparison_df.round(2))

## 12. Visualisation de la comparaison

In [ ]:
# Créer des visualisations comparatives
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Comparaison des Stratégies de Traitement', fontsize=16)

# 1. Nombre d'observations conservées
comparison_df.plot(x='Stratégie', y='Nb_Observations', kind='bar', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('Nombre d\'Observations Conservées')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Pourcentage de données conservées
comparison_df.plot(x='Stratégie', y='Pourcentage_Données_Conservées', kind='bar', ax=axes[0, 1], color='lightgreen')
axes[0, 1].set_title('Pourcentage de Données Conservées')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].set_ylabel('Pourcentage (%)')

# 3. Valeurs manquantes restantes
comparison_df.plot(x='Stratégie', y='Valeurs_Manquantes', kind='bar', ax=axes[0, 2], color='coral')
axes[0, 2].set_title('Valeurs Manquantes Restantes')
axes[0, 2].tick_params(axis='x', rotation=45)

# 4. Comparaison des moyennes pour une variable numérique
income_means = []
strategy_names = []
for name, data in strategies_results.items():
    if data is not None and 'income' in data.columns and len(data) > 0:
        income_means.append(data['income'].mean())
        strategy_names.append(name)

axes[1, 0].bar(strategy_names, income_means, color='gold')
axes[1, 0].set_title('Moyenne des Revenus par Stratégie')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].set_ylabel('Revenu moyen')

# 5. Distribution de l'âge pour quelques stratégies
strategies_to_plot = ['Original', 'Imputation_Moyenne', 'Imputation_KNN', 'Imputation_MICE']
for i, strategy in enumerate(strategies_to_plot):
    if strategy in strategies_results and 'age' in strategies_results[strategy].columns:
        axes[1, 1].hist(strategies_results[strategy]['age'].dropna(), alpha=0.5, label=strategy, bins=20)
axes[1, 1].set_title('Distribution de l\'Âge par Stratégie')
axes[1, 1].legend()
axes[1, 1].set_xlabel('Âge')
axes[1, 1].set_ylabel('Fréquence')

# 6. Boxplot des revenus par stratégie
income_data_for_box = []
labels_for_box = []
for name in ['Original', 'Imputation_Moyenne', 'Imputation_Médiane', 'Imputation_KNN']:
    if name in strategies_results and 'income' in strategies_results[name].columns:
        income_data_for_box.append(strategies_results[name]['income'].dropna())
        labels_for_box.append(name)

if income_data_for_box:
    axes[1, 2].boxplot(income_data_for_box, labels=labels_for_box)
    axes[1, 2].set_title('Distribution des Revenus par Stratégie')
    axes[1, 2].tick_params(axis='x', rotation=45)
    axes[1, 2].set_ylabel('Revenu')

plt.tight_layout()
plt.show()

## 13. Évaluation de l'impact sur un modèle de classification

In [ ]:
print("=" * 80)
print("ÉVALUATION DE L'IMPACT SUR UN MODÈLE DE CLASSIFICATION")
print("=" * 80)

model_results = []

# Tester chaque stratégie avec un modèle de classification
for strategy_name, data in strategies_results.items():
    if data is None or len(data) == 0 or 'target' not in data.columns:
        continue
    
    try:
        # Préparer les données
        X = data.drop(columns=['target'])
        y = data['target']
        
        # Vérifier s'il y a encore des valeurs manquantes
        if X.isnull().sum().sum() > 0:
            print(f"⚠️  {strategy_name}: Contient encore des valeurs manquantes, ignoré")
            continue
        
        # Encoder les variables catégorielles
        X_encoded = X.copy()
        encoders = {}
        for col in X_encoded.select_dtypes(include=['object']).columns:
            le = LabelEncoder()
            X_encoded[col] = le.fit_transform(X_encoded[col])
            encoders[col] = le
        
        # Encoder la variable cible
        le_target = LabelEncoder()
        y_encoded = le_target.fit_transform(y)
        
        # Vérifier qu'il y a assez de données pour faire un split
        if len(X_encoded) < 10:
            print(f"⚠️  {strategy_name}: Pas assez de données ({len(X_encoded)} observations)")
            continue
        
        # Diviser les données
        X_train, X_test, y_train, y_test = train_test_split(
            X_encoded, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
        )
        
        # Entraîner le modèle
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        
        # Prédictions et évaluation
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        model_results.append({
            'Stratégie': strategy_name,
            'Accuracy': accuracy,
            'Taille_Train': len(X_train),
            'Taille_Test': len(X_test),
            'Nb_Features': len(X.columns)
        })
        
        print(f"✅ {strategy_name}: Accuracy = {accuracy:.4f}")
        
    except Exception as e:
        print(f"❌ Erreur avec {strategy_name}: {e}")

# Afficher les résultats
if model_results:
    model_results_df = pd.DataFrame(model_results)
    print("\nRésultats de performance des modèles:")
    print(model_results_df.round(4))
    
    # Visualiser les performances
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    model_results_df.plot(x='Stratégie', y='Accuracy', kind='bar', ax=plt.gca(), color='lightblue')
    plt.title('Performance des Modèles par Stratégie')
    plt.ylabel('Accuracy')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    plt.scatter(model_results_df['Taille_Train'], model_results_df['Accuracy'], s=100, alpha=0.7)
    for i, row in model_results_df.iterrows():
        plt.annotate(row['Stratégie'], (row['Taille_Train'], row['Accuracy']), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    plt.xlabel('Taille du Set d\'Entraînement')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs Taille des Données')
    
    plt.tight_layout()
    plt.show()
else:
    print("Aucun résultat de modèle disponible.")

## 14. Recommandations et conclusions

In [ ]:
print("=" * 80)
print("RECOMMANDATIONS ET CONCLUSIONS")
print("=" * 80)

print("\n🎯 CHOIX DE LA STRATÉGIE EN FONCTION DU CONTEXTE:")
print("-" * 50)
print("1. SUPPRESSION DES LIGNES:")
print("   ✅ Quand: <5% de valeurs manquantes et MCAR (Missing Completely At Random)")
print("   ✅ Avantage: Simple, pas de biais d'imputation")
print("   ❌ Inconvénient: Perte de données, réduction de la puissance statistique")

print("\n2. SUPPRESSION DES COLONNES:")
print("   ✅ Quand: >50% de valeurs manquantes dans une colonne")
print("   ✅ Avantage: Élimine les variables peu fiables")
print("   ❌ Inconvénient: Perte d'information potentiellement utile")

print("\n3. IMPUTATION SIMPLE (Moyenne/Médiane/Mode):")
print("   ✅ Quand: Données MCAR avec distribution normale")
print("   ✅ Avantage: Rapide, facile à implémenter")
print("   ❌ Inconvénient: Réduit la variance, peut introduire des biais")

print("\n4. IMPUTATION KNN:")
print("   ✅ Quand: Relations locales importantes entre observations")
print("   ✅ Avantage: Préserve les patterns locaux")
print("   ❌ Inconvénient: Sensible aux outliers, computationnellement coûteux")

print("\n5. IMPUTATION ITÉRATIVE (MICE):")
print("   ✅ Quand: Relations complexes entre variables")
print("   ✅ Avantage: Modélise les interdépendances")
print("   ❌ Inconvénient: Plus complexe, peut ne pas converger")

print("\n\n📊 FACTEURS À CONSIDÉRER:")
print("-" * 30)
print("• Type de mécanisme de manquance:")
print("  - MCAR: Missing Completely At Random")
print("  - MAR: Missing At Random")
print("  - MNAR: Missing Not At Random")
print("\n• Proportion de valeurs manquantes")
print("• Taille du dataset")
print("• Nature des variables (numériques vs catégorielles)")
print("• Objectif de l'analyse (exploration vs prédiction)")
print("• Contraintes computationnelles")

print("\n\n🏆 RECOMMANDATIONS GÉNÉRALES:")
print("-" * 35)
print("1. Toujours analyser le pattern des valeurs manquantes AVANT de choisir une stratégie")
print("2. Tester plusieurs stratégies et comparer leurs impacts")
print("3. Documenter la stratégie choisie et ses justifications")
print("4. Considérer l'impact sur les analyses en aval")
print("5. Prévoir une validation croisée robuste")

print("\n✅ Exercice 2 terminé avec succès!")
print("📈 Vous maîtrisez maintenant les principales stratégies de traitement des valeurs manquantes.")